In [1]:
import os, gc, sys
import random

import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from simpletransformers.classification import ClassificationModel
import torch

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
SEED = 2020
BASE_PATH = '/home/toshiya/Workspace/learning/signate/SIGNATE_Student_Cup_2020'
TEXT_COL = "description"
TARGET = "jobflag"
NUM_CLASS = 4
N_FOLDS = 4

In [3]:
def metric_f1(labels, preds):
    return f1_score(labels, preds, average='macro')

In [4]:
def seed_everything(seed):
    """for reproducibility.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

In [5]:
train = pd.read_csv(os.path.join(BASE_PATH, 'data', "train.csv")).drop(['id'], axis=1)
train = train.rename(columns={TEXT_COL:'text', TARGET:'label'})
train['label'] -= 1

test = pd.read_csv(os.path.join(BASE_PATH, 'data', "test.csv"))
test = test.rename(columns={TEXT_COL:'text'}).drop(['id'], axis=1)

In [6]:
kfold = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
train['fold_id'] = -1
for fold, (train_idx, valid_idx) in enumerate(kfold.split(train.index, train['label'])):
    train.loc[train.iloc[valid_idx].index, 'fold_id'] = fold

X_train = train.loc[train['fold_id']!=0]
X_valid = train.loc[train['fold_id']==0]

In [7]:
os.makedirs(os.path.join(BASE_PATH, "result", "03_NLP_baseline"), exist_ok=True)
params = {
    "output_dir": os.path.join(BASE_PATH, "result", "03_NLP_baseline"),
    "max_seq_length": 128,
    "train_batch_size": 32,
    "eval_batch_size": 64,
    "num_train_epochs": 70,
    "learning_rate": 1e-4,
    "manual_seed":SEED,
    "save_eval_checkpoints": False,
    "save_steps": 6900,
    "save_model_every_epoch": False,
    "no_cache": True,
    "overwrite_output_dir": True
}
model = ClassificationModel('bert', 'bert-base-cased', num_labels=4,
                            args=params, use_cuda=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
model.train_model(X_train)

/home/toshiya/.pyenv/versions/3.8.5/envs/studentcup2020/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:281: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


/home/toshiya/.pyenv/versions/3.8.5/envs/studentcup2020/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


In [9]:
result, model_outputs, wrong_predictions = model.eval_model(X_valid, f1=metric_f1)
print(result)

/home/toshiya/.pyenv/versions/3.8.5/envs/studentcup2020/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:753: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(



{'mcc': 0.49885132810886523, 'f1': 0.6188720998376489, 'eval_loss': 3.4990211526552835}


In [10]:
y_pred, raw_outputs = model.predict(test['text'])
print(y_pred)


[3 2 2 ... 0 2 2]


In [11]:
test = pd.read_csv(os.path.join(BASE_PATH, "data", "test.csv"))
submit = pd.DataFrame({'index':test['id'], 'pred':y_pred+1})
submit

,index,pred
0,2931,4
1,2932,3
2,2933,3
3,2934,1
4,2935,3
...,...,...
1738,4669,1
1739,4670,3
1740,4671,1
1741,4672,3


In [12]:
submit.to_csv(os.path.join(BASE_PATH, "result", "03_NLP_baseline", "03_NLP_baseline.csv"), index=False, header=False)